#Hi Welcome to The NoteBook 
###We will implement tokenization API in keras to encode sentences then we will use embedding to make sense of tokens and classify using embeddings 
##Data used : IMDB from tensorflow datasets(tfds)


#Import


In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import io
print(tf.__version__)

2.2.0


#Load The dataset from tfds

In [4]:
imdb , info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

train_data,test_data = imdb["train"],imdb["test"]
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s,l in train_data:
  training_sentences.append(s.numpy().decode("utf8"))
  training_labels.append(l.numpy())

for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteD2C5XK/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteD2C5XK/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteD2C5XK/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


#Hyperparameter for training

In [5]:
vocab_size = 20000
embedding_dim = 32
max_length = 120
trunc_type = "post" #want to preserve the initial words
oov_token = "UNK"

In [11]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type,padding="post")

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

#Function to retrive words from index and get the sentence

In [12]:
def decode_review(text):
  """ it will give you the original text"""
  reverse_dictorary = dict([(value,key) for (key,value) in word_index.items()])
  return " ".join([reverse_dictorary.get(i,"UNK") for i in text])

#test
print(decode_review(padded[3]))
print(training_sentences[3])
#assert(decode_review(padded[3])==training_sentences[3])

this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm chair and mellow for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no UNK to cross no dangerous waters just a warm and witty UNK through new york life at its best a family film in every sense and one that deserves the praise it received UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK UNK
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and on

#Model Building

In [13]:
#difine the model
def model(vocab_size,embedding_dim,max_length):
  model = tf.keras.Sequential([
                              tf.keras.layers.Embedding(vocab_size,embedding_dim, input_length=max_length),
                              tf.keras.layers.Flatten(),
                              tf.keras.layers.Dense(12,activation="relu"),
                              tf.keras.layers.Dense(1,activation="sigmoid")

])
  model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])
  model.summary()
  return model

In [14]:
model = model(vocab_size,embedding_dim,max_length)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 32)           640000    
_________________________________________________________________
flatten (Flatten)            (None, 3840)              0         
_________________________________________________________________
dense (Dense)                (None, 12)                46092     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 13        
Total params: 686,105
Trainable params: 686,105
Non-trainable params: 0
_________________________________________________________________


#Training Parameters

In [15]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 8s 11ms/step - loss: 0.4605 - accuracy: 0.7676 - val_loss: 0.3581 - val_accuracy: 0.8407
Epoch 2/10
782/782 [==============================] - 8s 11ms/step - loss: 0.1531 - accuracy: 0.9467 - val_loss: 0.4392 - val_accuracy: 0.8173
Epoch 3/10
782/782 [==============================] - 8s 11ms/step - loss: 0.0202 - accuracy: 0.9968 - val_loss: 0.5053 - val_accuracy: 0.8227
Epoch 4/10
782/782 [==============================] - 8s 11ms/step - loss: 0.0034 - accuracy: 0.9998 - val_loss: 0.5661 - val_accuracy: 0.8232
Epoch 5/10
782/782 [==============================] - 8s 11ms/step - loss: 8.4467e-04 - accuracy: 1.0000 - val_loss: 0.6173 - val_accuracy: 0.8202
Epoch 6/10
782/782 [==============================] - 8s 10ms/step - loss: 3.9097e-04 - accuracy: 1.0000 - val_loss: 0.6598 - val_accuracy: 0.8192
Epoch 7/10
782/782 [==============================] - 8s 10ms/step - loss: 2.1692e-04 - accuracy: 1.0000 - val_loss: 0.6668 - val_accu

#scrap the weights

In [16]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(20000, 32)


understand it we give 20000 vocab size and 32 dimention for each of the weights

#Download weights for visualization

In [17]:
reverse_dictorary = dict([(value,key) for (key,value) in word_index.items()])
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_dictorary[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [18]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]


#Visualization:
follow [this link](https://projector.tensorflow.org/) and upload vec.tsv and meta.tsv to visualization use t-sne as decomposition method

#Written by : SATYAJIT MAITRA
## follow github: https://github.com/MachineLearningWithHuman/
## Email for work : ssatyajitmaitra@gmail.com
##learn with me on youtube: https://www.youtube.com/channel/UCiWd572-4LeH0IqJ5A7LavA/
### please give a star if it helps you 